# Homework and bake-off: Relation extraction using distant supervision

In [55]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Fall 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [2]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [57]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [58]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [59]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [60]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [95]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [62]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Baselines

### Hand-build feature functions

In [63]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [64]:
featurizers = [simple_bag_of_words_featurizer]

In [65]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [66]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.848      0.362      0.668        340       5716
author                    0.806      0.548      0.737        509       5885
capital                   0.600      0.221      0.447         95       5471
contains                  0.816      0.598      0.760       3904       9280
film_performance          0.796      0.570      0.738        766       6142
founders                  0.794      0.395      0.660        380       5756
genre                     0.622      0.165      0.400        170       5546
has_sibling               0.847      0.232      0.554        499       5875
has_spouse                0.894      0.327      0.663        594       5970
is_a                      0.667      0.217      0.472        497       5873
nationality               0.644      0.186      0.431        301       5677
parents     

Studying model weights might yield insights:

In [67]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.470 Taluks
     2.464 Córdoba
     2.324 Valais
     ..... .....
    -1.353 other
    -1.496 who
    -2.067 Earth

Highest and lowest feature weights for relation author:

     2.881 author
     2.664 books
     2.491 book
     ..... .....
    -2.294 1925
    -2.997 Daisy
    -3.366 1945

Highest and lowest feature weights for relation capital:

     3.604 capital
     1.699 km
     1.550 posted
     ..... .....
    -1.990 ~3.9
    -2.002 million
    -2.009 pop

Highest and lowest feature weights for relation contains:

     2.200 third-largest
     2.183 transferred
     2.088 affiliated
     ..... .....
    -2.584 band
    -2.603 who
    -2.676 2002

Highest and lowest feature weights for relation film_performance:

     4.163 starring
     3.764 opposite
     3.591 movie
     ..... .....
    -2.019 Iruvar
    -2.093 comedian
    -3.779 Mohabbatein

Highest and lowest feature weights for relation founders:

     3.918 f

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [68]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [69]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [70]:
print(type(glove_lookup))
print('glove_lookup has {} keys'.format(len(glove_lookup.keys())))
print('1st key: {}'.format(next(iter(glove_lookup.keys()))))
print('first value is an array of shape {}'.format(next(iter(glove_lookup.values())).shape))

<class 'dict'>
glove_lookup has 400000 keys
1st key: the
first value is an array of shape (300,)


In [71]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [72]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.900      0.450      0.750        340       5716
author                    0.788      0.438      0.679        509       5885
capital                   0.562      0.189      0.404         95       5471
contains                  0.649      0.408      0.581       3904       9280
film_performance          0.838      0.324      0.636        766       6142
founders                  0.752      0.239      0.527        380       5756
genre                     0.444      0.071      0.216        170       5546
has_sibling               0.852      0.255      0.580        499       5875
has_spouse                0.867      0.352      0.671        594       5970
is_a                      0.672      0.161      0.411        497       5873
nationality               0.613      0.189      0.423        301       5677
parents     

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [73]:
def run_svm_model_factory():

    ##### YOUR CODE HERE
    from sklearn.svm import SVC
    
    model_svc = lambda: SVC(kernel='linear')
    svc_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=featurizers,
        model_factory=model_svc,
        verbose=True)
    return svc_results


In [74]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."

In [75]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.779      0.332      0.614        340       5716
author                    0.728      0.615      0.702        509       5885
capital                   0.683      0.295      0.541         95       5471
contains                  0.792      0.599      0.744       3904       9280
film_performance          0.737      0.627      0.712        766       6142
founders                  0.742      0.439      0.652        380       5756
genre                     0.533      0.235      0.426        170       5546
has_sibling               0.796      0.242      0.547        499       5875
has_spouse                0.835      0.350      0.654        594       5970
is_a                      0.620      0.276      0.496        497       5873
nationality               0.566      0.199      0.414        301       5677
parents     

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [ ]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter):
    # Append these to the end of the keys you add/access in
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word + object_subject_suffix] += 1


    return feature_counter


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[directional_bag_of_words_featurizer],
    model_factory=model_factory,
    verbose=True)
nber_features = len(result['vectorizer'].get_feature_names())
print('The vectorizer has {} feature names'.format(nber_features))



In [77]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [78]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [79]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    def get_bigram_count(example, feature_counter):
        sentence = example.middle_POS.strip() + " </s>"
        prev_tag = "<s>"
        for word in sentence.split(' '):
            if not word:
                continue
            tag = word if word == "</s>" else word.split('/')[1]
            bigram = prev_tag + " " + tag
            prev_tag = tag
            feature_counter[bigram] += 1
        return feature_counter

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        feature_counter = get_bigram_count(ex, feature_counter)

    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        feature_counter = get_bigram_count(ex, feature_counter)
    return feature_counter


def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"

    ##### YOUR CODE HERE




def get_tags(s):
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[middle_bigram_pos_tag_featurizer],
    model_factory=model_factory,
    verbose=True)




relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.850      0.332      0.648        340       5716
author                    0.732      0.332      0.590        509       5885
capital                   0.517      0.158      0.355         95       5471
contains                  0.769      0.593      0.726       3904       9280
film_performance          0.721      0.435      0.637        766       6142
founders                  0.566      0.168      0.385        380       5756
genre                     0.604      0.171      0.401        170       5546
has_sibling               0.722      0.166      0.433        499       5875
has_spouse                0.764      0.261      0.551        594       5970
is_a                      0.593      0.161      0.386        497       5873
nationality               0.407      0.080      0.223        301       5677
parents     

In [80]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [81]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [97]:
from nltk.corpus import wordnet as wn

def synset_featurizer(kbt, corpus, feature_counter):
    """
    This feature function is just like simple_bag_of_words_featurizer except that it returns a list of Wordnet synsets
    derived from kbt.middle_POS. The synsets have been turned into a string so that we can use them as keys in a
    dictionary.
    """
    # Get all examples in corpus for kbt independently of the subject/object relative position
    all_examples = corpus.get_examples_for_entities(kbt.sbj, kbt.obj).copy()
    all_examples += corpus.get_examples_for_entities(kbt.obj, kbt.sbj).copy()
    for ex in all_examples:
        synsets = get_synsets(ex.middle_POS)
        for synset in synsets:
            feature_counter[synset] += 1
    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets
    associated with elements of `s`.
    """
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]

    # Converts the POS tags so that they can be used by WordNet.
    wt_wn = [[pair[0], convert_tag(pair[1])] for pair in wt]
    # list all "stringified" synsets associated to each [word, tag] in wt_wn
    synsets = [str(synset) for pair in wt_wn for synset in wn.synsets(pair[0], pos=pair[1])]
    return synsets


def convert_tag(t):
    """Converts tags so that they can be used by WordNet:

    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[synset_featurizer],
    model_factory=model_factory,
    verbose=True)



/Users/gdegournay/opt/anaconda3/envs/nlu/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.771      0.318      0.600        340       5716
author                    0.786      0.440      0.679        509       5885
capital                   0.590      0.242      0.458         95       5471
contains                  0.800      0.590      0.746       3904       9280
film_performance          0.776      0.556      0.719        766       6142
founders                  0.751      0.382      0.629        380       5756
genre                     0.448      0.176      0.342        170       5546
has_sibling               0.862      0.224      0.550        499       5875
has_spouse                0.857      0.303      0.628        594       5970
is_a                      0.594      0.223      0.446        497       5873
nationality               0.478      0.143      0.325        301       5677
parents     

In [90]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [91]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. We also ask that you report the best score your system got during development, just to help us understand how systems performed overall.

In [7]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM 
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
#
# The split used to train and test the classifier is the default split proposed in the notebook:
#     split_fracs=[0.01, 0.79, 0.20].
#
# The classifier used is the LogisticRegression with the following parameters:
#     fit_intercept=True, 
#     solver='liblinear', 
#     multi_class='auto'.
#
# Featurizer:
# The Featurizer used is a modified version of simple_bag_of_words_featurizers, 
# including bigrams and trigrams as well. A bigram being "word1 word2" and a trigram
# "word1 word2 word3". It also uses a directional token feature - a token being a unigram,
# a bigram or a trigram - similar to the one used in the notebook by adding a subject-object
# suffix or a object-subject suffix to all the tokens.
# The function used for that purpose is ngrams_featurizer with parameters type='trigrams' 
# and directional=True.

# My peak score was: 0.649
if 'IS_GRADESCOPE_ENV' not in os.environ:
    
    import numpy as np
    import os
    import rel_ext
    from sklearn.linear_model import LogisticRegression
    import utils
    
    utils.fix_random_seeds()
    rel_ext_data_home = os.path.join('data', 'rel_ext_data')
    corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))
    kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))
    dataset = rel_ext.Dataset(corpus, kb)

    def get_bigrams_count(sentence, feature_counter, suffix):
        new_sentence = sentence + " </s>"
        prev_word = "<s>"
        for word in new_sentence.strip().split(' '):
            if not word:
                continue
            bigram = prev_word + " " + word
            prev_word = word
            if word != "</s>":
                feature_counter[word + suffix] += 1
            feature_counter[bigram + suffix] += 1
        return feature_counter


    def get_trigrams_count(sentence, feature_counter, suffix):
        new_sentence = sentence + " </s> </s>"
        prev_word = "<s>"
        prev_bigram = "<s> <s>"
        for word in new_sentence.strip().split(' '):
            if not word:
                continue
            trigram = prev_bigram + " " + word
            bigram = prev_word + " " + word
            prev_bigram = bigram
            prev_word = word

            if word not in ["<s>", "</s>"]:
                feature_counter[word + suffix] += 1
            if bigram not in ["<s> <s>", "</s> </s>"]:
                feature_counter[bigram + suffix] += 1
            feature_counter[trigram + suffix] += 1
        return feature_counter


    def ngrams_featurizer(kbt, corpus, feature_counter, type, directional=False):
        """
        Modified version of simple_bag_of_words_featurizers, include bigrams or trigrams as well depending on 'type'.
        :param kbt:
        :param corpus:
        :param feature_counter:
        :param type: 'bigrams' or 'trigrams
        :param directional: indicates if we take into account the order of subject and object.
        :return:
        """
        subject_object_suffix = "_SO" if directional else ""
        object_subject_suffix = "_OS" if directional else ""
        if type == 'bigrams':
            ngrams = get_bigrams_count
        elif type == 'trigrams':
            ngrams = get_trigrams_count
        else:
            raise Exception('type needs to be bigrams or trigrams.')

        for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
            feature_counter = ngrams(ex.middle, feature_counter, subject_object_suffix)

        for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
            feature_counter = ngrams(ex.middle, feature_counter, object_subject_suffix)
        return feature_counter

    
    def rel_ext_model(dataset):
        """
        :param dataset: rel_ext.Dataset
        :return:
        dictionary: {
            'featurizers': [list of feature functions],
            'vectorizer': DictVectorizer used to convert feature dicts in matrices
            'models': {relation: fitted model}
            'all_relations': [list of relations considered]
            'vectorize': bool # indicates if DictVectorizer is needed.
        } 
        """
        splits = dataset.build_splits(split_fracs=[0.01, 0.79, 0.20])
        lr = lambda: LogisticRegression(fit_intercept=True, solver='liblinear', multi_class='auto')
        grams3_direct_featurizer = lambda *args: ngrams_featurizer(*args, type='trigrams', directional=True)

        baseline_results = rel_ext.experiment(
            splits,
            train_split='train',
            test_split='dev',
            featurizers=[grams3_direct_featurizer],
            model_factory=lr,
            verbose=True)
        return baseline_results
    
    model_result = rel_ext_model(dataset)
    

# STOP COMMENT: Please do not remove this comment.

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.890      0.382      0.703        340       5716
author                    0.902      0.635      0.832        509       5885
capital                   0.793      0.242      0.545         95       5471
contains                  0.782      0.761      0.778       3904       9280
film_performance          0.875      0.701      0.833        766       6142
founders                  0.831      0.413      0.691        380       5756
genre                     0.753      0.359      0.617        170       5546
has_sibling               0.902      0.240      0.582        499       5875
has_spouse                0.906      0.355      0.691        594       5970
is_a                      0.716      0.294      0.556        497       5873
nationality               0.765      0.302      0.586        301       5677
parents     

## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [ ]:
# Enter your bake-off assessment code in this cell.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE


In [ ]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported
# by the code above. Please enter only a number between
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
